In [1]:
# Seed
import torch
import numpy as np
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [15]:
import torch
import torch.nn as nn

class VGG11(nn.Module):
  def __init__(self,num_classes=1000):
    super(VGG11,self).__init__()

    self.relu = nn.ReLU(inplace=True)

    self.conv1 = nn.Conv2d(3,64,kernel_size=3,padding=1) # 3채널 input -> 64채널 output
    self.bn1 = nn.BatchNorm2d(64)
    self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2) # 공간(spatial)정보가 가로 세로 1/2배씩 크기가 줄어듬

    self.conv2 = nn.Conv2d(64,128,kernel_size=3,padding=1)
    self.bn2 = nn.BatchNorm2d(128)
    self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv3_1 = nn.Conv2d(128,256,kernel_size=3,padding=1)
    self.bn3_1 = nn.BatchNorm2d(256)
    self.conv3_2 = nn.Conv2d(256,256,kernel_size=3,padding=1)
    self.bn3_2 = nn.BatchNorm2d(256)
    self.pool3 = nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv4_1 = nn.Conv2d(256,512,kernel_size=3,padding=1)
    self.bn4_1 = nn.BatchNorm2d(512)
    self.conv4_2 = nn.Conv2d(512,512,kernel_size=3,padding=1)
    self.bn4_2 = nn.BatchNorm2d(512)
    self.pool4 = nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv5_1 = nn.Conv2d(512,512,kernel_size=3,padding=1)
    self.bn5_1 = nn.BatchNorm2d(512)
    self.conv5_2 = nn.Conv2d(512,512,kernel_size=3,padding=1)
    self.bn5_2 = nn.BatchNorm2d(512)
    self.pool5 = nn.MaxPool2d(kernel_size=2,stride=2)

    self.fc1 = nn.Linear(512*7*7,4096) # 채널이 512 , 입력 이미지 사이즈 224x224 -> 7x7 (maxpooling이 5번 적용되었으므로)
    self.dropout1 = nn.Dropout() # 기본 p=0.5

    self.fc2 = nn.Linear(4096,4096)
    self.dropout2 = nn.Dropout()

    self.fc3 = nn.Linear(4096,num_classes)
  
  def forward(self,x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.pool1(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.pool2(x) 

    x = self.conv3_1(x)
    x = self.bn3_1(x)
    x = self.relu(x) # 활성화 함수 이전에는 bn 필수
    x = self.conv3_2(x)
    x = self.bn3_2(x)
    x = self.relu(x)
    x = self.pool3(x)

    x = self.conv4_1(x)
    x = self.bn4_1(x)
    x = self.relu(x)
    x = self.conv4_2(x)
    x = self.bn4_2(x)
    x = self.relu(x)
    x = self.pool4(x)

    x = self.conv5_1(x)
    x = self.bn5_1(x)
    x = self.relu(x)
    x = self.conv5_2(x)
    x = self.bn5_2(x)
    x = self.relu(x)
    x = self.pool5(x)    

    x = torch.flatten(x,1)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.dropout1(x)

    x = self.fc2(x)
    x = self.relu(x)
    x = self.dropout2(x)

    x = self.fc3(x)
    return x
    

In [16]:
model = VGG11(num_classes=1000)

x = torch.randn((5,3,224,224)) # 배치수, 채널 ,w ,h

out = model(x)

print(out.shape)

torch.Size([5, 1000])


In [19]:
# Torchvision 을 통한 pretrained 모델 사용
import torchvision.models as models

model_1 = VGG11(num_classes=1000) # 내가 만든 모델
model_2 = models.vgg11(pretrained=True) # torchvision.models 모델

print(model_1)
print(model_2)

VGG11(
  (relu): ReLU(inplace=True)
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3_1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3_2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4_1): Conv2d(256, 51